# Master Option Selector

In [11]:
def ScreenerOptionMaster(master_df, volScreen, maScreen, rsiScreen, priceScreen, industryScreen, historical_datas_fromInterface, minVolume, minPrice):
  # Keep passing around data frame
  import pandas as pd
  temp_df = master_df.copy()
  temp_df =   temp_df.rename(columns={'Market Cap Size':'MarketCapSize'})

  historicalDataGathered = False #

  # Print Num Stocks in Master List
  startTickersNumber = temp_df.shape[0]

# Industry Screener
  if industryScreen == True:
    print("Running Industry Screener")
    temp_df = industryScreener(temp_df)
  tickersAfterIndustry = temp_df.shape[0]

# Gathering Historical Data
#
#
# Call after screeners used that don't require data (industry screener is one)

# if historical data has not been gathered yet, get it now
  if historical_datas_fromInterface == None:
    historical_datas = getHistoricalData(temp_df)
  else:
    historical_datas = historical_datas_fromInterface

# Price Screener
  if priceScreen == True:
    print("Running Price Screener")
    temp_df = oneDollarScreener(temp_df, historical_datas, minPrice)
  tickersAfterPrice = temp_df.shape[0]


# Volume Screener
  if volScreen == True:
    print("Running Volume Screener")
    temp_df = volumeScreener(temp_df, historical_datas, minVolume)
  tickersAfterVol = temp_df.shape[0]



# Printing early results to console
  print(f"Number Stocks Beginning With: {startTickersNumber}")

  if industryScreen == True:
    print(f"Number Stocks After Industry Screener: {tickersAfterIndustry}")
  if priceScreen == True:
    print(f"Number Stocks After Price Screener: {tickersAfterPrice}")
  if volScreen == True:
    print(f"Number Stocks After Volume Screener: {tickersAfterVol}")

  return temp_df, historical_datas

# $1 Price Screener

In [10]:
def oneDollarScreener(initial_df, historical_datas, minPrice):
  from yahoofinancials import YahooFinancials
  from yahoo_fin.stock_info import get_data
  from yahoo_finance import Share
  import pandas as pd
  import numpy as np

  prices_df = pd.DataFrame(columns=['Symbol','Price(avg)'])

  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()
  offset = BMonthEnd(-1)
  offsetFirst = BMonthBegin(-13)

  # Get first mo. first day, last mo. last day
  lastDayLastMonth = todayD+offset
  firstDayFirstMonth = todayD+offsetFirst


  # Create ticker list from prices_df symbols
  ticker_list = initial_df['Symbol'].to_list()

  # Historical_datas holds the year worth of data. 
  # Here we will be iterating over the tickers provided in the parameters df. Price screener will screen to get their prices from one year ago, till today.
  for ticker in ticker_list:
    try:
      tickerMean = historical_datas[ticker]['close'].loc[f"{firstDayFirstMonth.year}-{firstDayFirstMonth.month}-{firstDayFirstMonth.day}" : f"{todayD.year}-{todayD.month}-{todayD.day}"].mean()
      if tickerMean >= minPrice:
        prices_df.loc[-1]= [ticker, tickerMean]
        prices_df.index = prices_df.index + 1
        prices_df = prices_df.sort_index()
      else:
        prices_df.loc[-1]= [ticker, ""]
        prices_df.index = prices_df.index + 1
        prices_df = prices_df.sort_index()
    except:
      print(f"error with {ticker}")
  

  # Drop na's
  prices_df.dropna()



  # Create new data frame and merge with the given data frame
  initial_df_copy = initial_df.copy()
  price_screened_merge_df = pd.merge(initial_df_copy, prices_df, on=['Symbol'])
  price_screened_merge_df.dropna()


  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  # clear_output()

  return price_screened_merge_df

# Volume Screener

In [9]:
def volumeScreener(initial_df, historical_datas, minVolume):
  # Testing putting these in Interface script
  #!pip install yfinance
  #!pip install yahoofinancials
  #!pip install yahoo-finance
  #!pip install yahoo_fin
  #!pip install yahoo_fin --upgrade
  #!pip install requests_html
  #!pip install gspread
  #!pip install XlsxWriter
  from yahoofinancials import YahooFinancials
  from yahoo_fin.stock_info import get_data
  from yahoo_finance import Share
  import pandas as pd
  import numpy as np

  ticker_list = initial_df['Symbol'].to_list()


  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()
  offset = BMonthEnd(-1)
  offsetFirst = BMonthBegin(-13)

  # This will be last day of the previous month
  #lastDayLastMonth = todayD+offset

  # This will be one year ago for current month
  firstDayFirstMonth = todayD+offsetFirst


  # Create new data frame to hold and start building a complete data frame from all the stocks on the list
  full_df = pd.DataFrame()
  tickerListByVolume = []
  tickerWithNotEnoughData = []

  for ticker in ticker_list:
    try:
            # Here we are getting historical data from one year ago until today.
      full_df[ticker] = historical_datas[ticker]['volume'].loc[f"{firstDayFirstMonth.year}-{firstDayFirstMonth.month}-{firstDayFirstMonth.day}" : f"{todayD.year}-{todayD.month}-{todayD.day}"]
      #print(f"completed {ticker}")
    except:
      tickerWithNotEnoughData.append(ticker)



  # Dropping those stocks with volumes less than 100k (avg)
  volGreaterThan = full_df.copy()
  volGreaterThan.drop([col for col, val in volGreaterThan.mean().iteritems() if val < minVolume], axis=1, inplace=True)


  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()


  # Create new data frame and merge with the given data frame
  volTickerList = volGreaterThan.columns.to_list()
  tickerVolumesList = volGreaterThan.mean()
  volume_df = pd.DataFrame(list(zip(volTickerList, tickerVolumesList)))
  volume_df.columns = ['Symbol', 'Volume(avg)']
  master_df_copy = initial_df.copy()
  inner_merge_df = pd.merge(master_df_copy, volume_df, on=['Symbol'])
  inner_merge_df.dropna()

  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()

  return inner_merge_df

# Industry Screener

In [12]:
def industryScreener(initialPD):
  # List of industries to keep
  import pandas as pd
  list_to_keep = ['Agricultural Chemicals','Aluminum','Engineering & Construction','Environmental Services','Home Furnishings','Homebuilding','Major Chemicals','Military/Government/Technical',
                  'Mining &; Quarrying of Nonmetallic Minerals (No Fuels)','Package Goods/Cosmetics','Paints/Coatings','Precious Metals','Specialty Chemicals','Steel/Iron Ore','Telecommunications Equipment',
                  'Aerospace','Auto Manufacturing','Auto Parts:O.E.M.','Automotive Aftermarket','Biotechnology: Laboratory Analytical Instruments','Building Materials','Construction/Ag Equipment/Trucks',
                  'Electrical Products','Electronic Components','Engineering & Construction','Fluid Controls','Industrial Machinery/Components','Industrial Specialties','Medical Specialities',
                  'Metal Fabrications','Specialty Chemicals','Telecommunications Equipment','Automotive Aftermarket','Building Products','Consumer Electronics/Appliances','Consumer Specialties',
                  'Diversified Electronic Products','Electrical Products','Electronic Components','Miscellaneous manufacturing industries','Specialty Chemicals','Apparel','Beverages (Production/Distribution)',
                  'Consumer Electronics/Appliances','Consumer Specialties','Packaged Foods','Plastic Products','Recreational Products/Toys','Specialty Foods','Telecommunications Equipment','Advertising','Broadcasting',
                  'Building operators','Catalog/Specialty Distribution','Consumer Specialties','Department/Specialty Retail Stores','Diversified Commercial Services','Hotels/Resorts',
                  'Other Consumer Services','Other Specialty Stores','Professional Services','Restaurants','Telecommunications Equipment','Transportation Services','Business Services',
                  'Finance: Consumer Services','Biotechnology: Biological Products (No Diagnostic Substances)','Biotechnology: Commercial Physical & Biological Research',
                  'Biotechnology: Electromedical & Electrotherapeutic Apparatus','Biotechnology: In Vitro & In Vivo Diagnostic Substances','Industrial Specialties','Major Pharmaceuticals',
                  'Medical/Dental Instruments','Misc Health and Biotechnology Services','Business Services','Industrial Machinery/Components','Multi-Sector Companies','Advertising','Computer Communications Equipment',
                  'Computer Manufacturing','Computer peripheral equipment','Computer Software: Prepackaged Software','Computer Software: Programming Data Processing','Diversified Commercial Services',
                  'EDP Services','Electrical Products','Electronic Components','Industrial Machinery/Components','Professional Services','Radio And Television Broadcasting And Communications Equipment',
                  'Semiconductors','Air Freight/Delivery Services','Marine Transportation']

  stocks_df = initialPD.copy()
  try:
    stocks_df = stocks_df.loc[stocks_df['Industry'].isin(list_to_keep)]  
  except:
    print("Name industry_x")
  
  try:
    stocks_df = stocks_df.loc[stocks_df['Industry_x'].isin(list_to_keep)] 
  except:
    print("Error in list")

    
  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()

  return stocks_df

# Get Historical Data

In [8]:
def getHistoricalData(initial_df):
  # Imports
  from yahoofinancials import YahooFinancials
  from yahoo_fin.stock_info import get_data
  from yahoo_finance import Share
  import pandas as pd
  import numpy as np
    # Get Dates
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()

  # Get Data
  ticker_list = initial_df['Symbol']
  historical_datas = {}
  failedList = []
  for ticker in ticker_list:
      try:
        historical_datas[ticker] = get_data(ticker, start_date = '01/01/2020' , end_date = f'{todayD.month}/{todayD.day}/{todayD.year}')
        #print(f"saved {ticker}")
      except:
        failedList.append(ticker)


  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()

  
  return historical_datas